Финальное задание по практике
--------------------

### Цель

Ознакомится с базовыми алгоритмами и базовыми инструментами машинного обучения для анализа и классификации текста.

### Задание

1) Получить категоризованные данные с <b>Facebook API</b><br>
2) Сохранить полученные данные<br>
3) Токенизация данных<br>
4) Удаление стоп-слов<br>
5) Стемминг<br>
6) Поиск паттерна имя-фамилия в тексте<br>
7) Формирование списка топ <i>100</i> важных токенов по каждой категории<br>
8) Категоризация новых текстов по наличию в них этих топ <i>100</i> токенов<br>
9) Оценка качества<br>

Кодировка для кода на <b>Python</b>.

In [1]:
# coding=utf-8

Для измирения времени работы некоторых участоков подключим модуль <b>"time"</b>.

In [2]:
import time as t

# вернет: текущее время в миллисекундах
def time(): return int(round(t.time() * 1000))

## Хранение данных

Для хранения данных была выбрана база данных <b>Elasticsearch</b>, которая предоставляет удобный интерфейс взаимодействия, а так же кроссплатформенность при работе с ней, за счет того, что взаимодействие осуществляется по HTTP протоколу.

Константы работы с базой данных <b>Elasticsearch</b>:

In [3]:
ES_INDEX = "fb_group_posts"                 # Аналог базы данных в сравнении с реляционными БД, под ним будут
                                            # хранится все данные, необходимые по заданию
ES_POST_DOC_TYPE = "post"                   
ES_NAME_RELATION_DOC_TYPE = "name_relation"

ES_BULK_ACTIONS_SIZE = 500                  # Размер пака данных отсылаемых за раз в elasticsearch, 
                                            # необходимо для оптимизации по скорости

Работа с базой данных не относится к заданию напрямую, по-этому описанию работе с ней уделено меньше внимания. Вся работа с <b>Elasticsearch</b> скрыта в ниже описанном классе <b>FacebookDBHelper</b>.

In [4]:
from elasticsearch import Elasticsearch
from elasticsearch.exceptions import TransportError
from elasticsearch import helpers
from copy import deepcopy


# Дата-класс, который описывает отношение наличия имени в тексте поста
# @param: fl_name - Имя Фамилия
# @param: post_id - id поста, в тексте которого содержится это имя и фамилия
class FLNameData(object):
    def __init__(self, fl_name, post_id):
        self.fl_name = fl_name
        self.post_id = post_id


class FacebookDBHelper(object):
    def __init__(self):
        self.es = Elasticsearch()
    
    def save_posts(self, group_name, group_domain, posts):
        actions = []

        for post in posts:
            if 'message' in post.keys():
                action = {
                    "_index": ES_INDEX,
                    "_type": ES_POST_DOC_TYPE,
                    "_id": post['id'],
                    "_source": {
                        "message": post['message'],
                        "group_name": group_name,
                        "group_domain": group_domain
                    }
                }

                actions.append(action)

        self.__bulk_insert(actions)
        
    def save_name_relations(self, relations):
        actions = []

        for relation in relations:
            action = {
                "_index": ES_INDEX,
                "_type": ES_NAME_RELATION_DOC_TYPE,
                "_source": {
                    "fl_name": relation.fl_name,
                    "post_id": relation.post_id
                }
            }

            actions.append(action)

        self.__bulk_insert(actions)

    def __bulk_insert(self, actions):
        actions_list = split_list(list(actions), ES_BULK_ACTIONS_SIZE)

        for acts in actions_list:
            helpers.bulk(self.es, acts)

    def get_post_by_id(self, id):
        return self.__get(doc_type=ES_POST_DOC_TYPE, id=id)

    def get_all_posts(self):
        return self.__get_all(doc_type=ES_POST_DOC_TYPE)

    def get_all_name_relations(self, doc_type):
        relations = self.__get_all(doc_type=doc_type)

        # noinspection PyTypeChecker
        return [FLNameData(fl_name=r['_source']['fl_name'], post_id=r["_id"]) for r in relations]

    def __get(self, doc_type, id):
        return self.es.get(index=ES_INDEX, doc_type=doc_type, id=id)

    def __get_all(self, doc_type, body=None):
        if body is None:
            body = {}

        result = []

        page = self.es.search(
            index=ES_INDEX,
            doc_type=doc_type,
            scroll='2m',
            search_type='scan',
            size=1000,
            body=body)

        scroll_id = page['_scroll_id']
        scroll_size = page['hits']['total']

        while scroll_size > 0:
            page = self.es.scroll(scroll_id=scroll_id, scroll='2m')

            scroll_id = page['_scroll_id']
            scroll_size = len(page['hits']['hits'])

            result.extend(page['hits']['hits'])

        return result

    def get_all_sources(self, doc_type):
        posts = self.get_all_posts()
        result = []

        for post in posts:
            if "_source" in post.keys():
                # noinspection PyTypeChecker
                result.append(post["_source"])

        return result

    def get_all_messages(self):
        sources = self.get_all_sources(doc_type=ES_POST_DOC_TYPE)
        result = []

        for source in sources:
            if 'message' in source.keys():
                # noinspection PyTypeChecker
                result.append(source["message"])

        return result

    def delete_all_posts(self):
        return delete_by_doc_type(
            es=self.es,
            index=ES_INDEX,
            type_=ES_POST_DOC_TYPE)

    def delete_all_name_relations(self):
        return delete_by_doc_type(
            es=self.es,
            index=ES_INDEX,
            type_=ES_NAME_RELATION_DOC_TYPE)

    def get_messages_by_domain(self, domain):
        posts = self.__get_all(doc_type=ES_POST_DOC_TYPE, body={
            "query": {
                "match": {
                    "group_domain": {
                        "query": domain,
                        "operator": "and"
                    }
                }
            }
        })
        
        return [p['_source']['message'] for p in posts]

    def get_name_relations_by_fl(self, fl_name):
        return self.__get_all(doc_type=ES_NAME_RELATION_DOC_TYPE, body={
            "query": {
                "match": {
                    "fl_name": {
                        "query": fl_name,
                        "operator": "and"
                    }
                }
            }
        })
    
    def get_all_domains(self):
        response = self.es.search(index=ES_INDEX, doc_type=ES_POST_DOC_TYPE, body={
            "size": 0,
            "aggs": {
                "langs": {
                    "terms": {
                        "field": "group_domain",
                    }
                }
            }
        })

        result = []

        for bucket in response['aggregations']['langs']['buckets']:
            result.append(bucket['key'])

        return result


def delete_by_doc_type(es, index, type_):
    try:
        count = es.count(index, type_)['count']
        max_count = 5000

        if not count:
            return 0

        tmp_count = count

        while tmp_count > 0:
            tmp_count -= max_count

            response = es.search(
                index=index,
                filter_path=["hits.hits._id"],
                body={"size": max_count,
                      "query": {
                          "filtered": {
                              "filter": {
                                    "type": {"value": type_}
                              }
                          }
                      }})

            if not response:
                return 0

            ids = [x["_id"] for x in response["hits"]["hits"]]

            if not ids:
                return 0

            bulk_body = [
                '{{"delete": {{"_index": "{}", "_type": "{}", "_id": "{}"}}}}'.format(index, type_, x)
                for x in ids]

            es.bulk('\n'.join(bulk_body))
            es.indices.flush_synced([index])

        return count
    except TransportError as ex:
        print("Elasticsearch error: " + ex.error)
        raise ex
        
def split_list(list_, count_):
    result = []

    if len(list_) == 0:
        return result

    if len(list_) == count_:
        result.append(list_)

        return result

    steps = len(list_) / count_
    tmp_list = deepcopy(list_)

    for i in range(0, steps):
        result.append(tmp_list[0: count_])
        tmp_list = tmp_list[count_: len(tmp_list)]

    if len(tmp_list) != 0:
        result.append(tmp_list)

    return result


Экземпляр класса <b>FacebookDBHelper</b>, через который мы будем взаимодействовать с <b>Elasticsearch</b> во всей работе.

In [5]:
fdb = FacebookDBHelper()

## Загрузка постов с facebook.com

Для работы с fecebook.com я выбрал <b>Facebook Graph API</b>, <b>SDK</b> предоставляющее выскоуровневый интерфейс.

Константы для работы с <b>Facebook API</b>:

In [6]:
# Токен зарегестрированного приложения в консоли разработчика на facebook.com
FACEBOOK_TOKEN = "1769775703259571|736fc7f9c5dc31707d40709a1d37813b"

# Кол-во постов загружаемых с одной группы на facebook.com
FACEBOOK_POSTS_COUNT = 4000

Сущьность описывающая заведомо известные данные о группе в <b>Facebook</b>:
* Имя
* Id на facebook.com
* Домен (класс) тематики группы

In [7]:
class Group(object):
    def __init__(self, name, id, domain):
        self.name = name
        self.id = id
        self.domain = domain

Список групп, которые будут использованы для работы (для загрузки постов):

In [8]:
groups = [
    Group(name="CNN Politics", id="219367258105115", domain="politics"),
    Group(name="SinoRuss", id="1565161760380398", domain="politics"),
    Group(name="Politics & Sociology", id="1616754815303974", domain="politics"),
    Group(name="CNN Money", id="6651543066", domain="finances"),
    Group(name="MoneyMagazine", id="119930514707114", domain="finances"),
    Group(name="MTV", id="7245371700", domain="music"),
    Group(name="MTV UK", id="15713980389", domain="music"),
    Group(name="CNET", id="7155422274", domain="tech"),
    Group(name="TechCrunch", id="8062627951", domain="tech"),
    Group(name="TechCrunchEurope", id="279044985158", domain="tech"),
    Group(name="TechInsider", id="352751268256569", domain="tech"),
    Group(name="Sport Addicts", id="817513368382866", domain="sport"),
    Group(name="Pokemon GO", id="1745029562403910", domain="pokemon_go")
]

Для работы с Facebook будем использовать экземпляр класса <b>GraphAPI</b>.

In [9]:
from facebook import GraphAPI


graph = GraphAPI(access_token=FACEBOOK_TOKEN)

Так же нам понадобится модуль для совершения <b>HTTP</b> запросов.

In [10]:
import requests

Функция загружающая последовательно посты пока не достигнет предела группы или ограничения по кол-ву (константа <b>FACEBOOK_POSTS_COUNT</b>).

In [11]:
def load_next_posts(posts, max_count):
    result = []
    count = 0

    while True:
        if count > max_count: # Если зугрузили необходимое кол-во то прекращаем работу
            break

        try:
            for post_data in posts['data']: # Под ключем 'data' хранится список постов в паке
                keys = post_data.keys()     # Так как мы нам нужны сами сообщения
                                            
                
                if 'message' in keys:       # Добавляем в результат только те у которых есть
                    result.append(post_data)# текстовое сообщение (смотрим поналичию ключа 'message')

            # Меняем размер запрашиваемого пака (страницы) с постами, в уже сформированном запросе 
            # от Facebook Graph API.
            request = posts['paging']['next'].replace("limit=25", "limit=100")

            # Выполняем запрос на получение следующей страницы с вопросами
            s_time = time()
            posts = requests.get(request).json()
            f_time = time()
            
            posts_count = len(posts['data'])
            count += posts_count
            
            print "Время загрузки пака постов ->", (f_time - s_time), "|", "кол-во:", posts_count, "|", "всего:", count
        except KeyError:
            break

    print "Общее кол-во загруженных постов группы ->", count

    return result

Используя ранее описанный список постов заполняем нашу базу данных.

In [12]:
for group in groups:
    # Получаем данные о группе по ее id на facebook.com
    s_time = time()
    group_json = graph.get_object(group.id)
    f_time = time()
    
    print 'Время загрузки данных [', group_json['name'], '] группы ->', (f_time - s_time)
    
    # Получаем первый пак постов, используем уже для этого метод "get_connections"
    # который, грубо говоря, дает возможность запрашивать списки
    s_time = time()
    first_posts_pack = graph.get_connections(group_json['id'], 'feed')
    f_time = time()
    
    print 'Время загрузки первого пака постов ->', (f_time - s_time)
    
    # Последовательно загружаем все последующие посты
    posts = load_next_posts(posts=first_posts_pack, max_count=FACEBOOK_POSTS_COUNT)
    
    # Сохраняем все в базу данных
    s_time = time()
    fdb.save_posts(group.name, group.domain, posts)
    f_time = time()
    
    print "Время сохранение постов в БД ->", (f_time - s_time)
    print "-------------------------"

Время загрузки данных [ CNN Politics ] группы -> 748
Время загрузки первого пака постов -> 671
Время загрузки пака постов -> 774 | кол-во: 100 | всего: 100
Время загрузки пака постов -> 927 | кол-во: 100 | всего: 200
Время загрузки пака постов -> 1753 | кол-во: 100 | всего: 300
Время загрузки пака постов -> 1723 | кол-во: 100 | всего: 400
Время загрузки пака постов -> 6548 | кол-во: 100 | всего: 500
Время загрузки пака постов -> 1521 | кол-во: 100 | всего: 600
Время загрузки пака постов -> 1411 | кол-во: 100 | всего: 700
Время загрузки пака постов -> 1585 | кол-во: 100 | всего: 800
Время загрузки пака постов -> 1545 | кол-во: 100 | всего: 900
Время загрузки пака постов -> 1631 | кол-во: 100 | всего: 1000
Время загрузки пака постов -> 1645 | кол-во: 100 | всего: 1100
Время загрузки пака постов -> 1552 | кол-во: 100 | всего: 1200
Время загрузки пака постов -> 1696 | кол-во: 100 | всего: 1300
Время загрузки пака постов -> 1446 | кол-во: 100 | всего: 1400
Время загрузки пака постов -> 1554

## Поиск паттерна (шаблона) "Имя Фамилия"

Решение построено на том, что бы разбить поиск на  два основных этапа:
1. Подготовить данные к поиску
2. Выполнить сам поиск

Подготовка в данном случае - это поиск всех пар <b>"имя фамилия"</b> заранее и кеширование свзяей <b>"имя фамилия" -> "id поста"</b> в базе данных. Это необходимо, что бы не осуществлять поиск в каждом посте при каждом запросе, а просто искать совпадения в уже найденных.

### Подготовка данных к поиску

Поиск имен выполнен с использованием регулярных выражений.

In [13]:
import re

# Регулярное выражение для "Имя Фамилия"
FIRST_LAST_NAME_PATTERN = "[A-Z]{1}[a-z]+\s+[A-Z]{1}[a-z]+"

Далее будем пользоваться уже сохраненными данными (текст постов, которые мы сохранили в базу данных <b>Elasticsearch</b>).

In [14]:
posts = fdb.get_all_posts() # Вытаскиваем все посты

Теперь можем выполнить поиск имен по шаблону.

In [15]:
name_relations = []

s_time = time()

for post in posts:
    message = post['_source']['message']
    names = re.findall(FIRST_LAST_NAME_PATTERN, message)

    if names:
        id = post['_id']

        for name in names:
            name = name.replace(".", "")
            relation = FLNameData(fl_name=name, post_id=id)
            name_relations.append(relation)

f_time = time()

print "Время поиска имен по регулярному выражению ->", (f_time - s_time)
print "Кол-во найденных совпадений по регулярному выраженияю:", len(name_relations)

Время поиска имен по регулярному выражению -> 373
Кол-во найденных совпадений по регулярному выраженияю: 40118


Поскольку все отношения будут хранить в базе данных, а предыдущим шагом были найдены пары "имя фамилия" во всех постах, то и очистим существующие связи, если они там есть.

In [16]:
s_time = time()
deleted_posts = fdb.delete_all_name_relations()
f_time = time()

print "Время удаления старых отношений 'сообщение <=> имя' ->", (f_time - s_time)
print "Удалено старых отношений:", deleted_posts

Время удаления старых отношений 'сообщение <=> имя' -> 8516
Удалено старых отношений: 30340


In [17]:
s_time = time()
fdb.save_name_relations(name_relations)
f_time = time()

print "Время сохранения новых отношений 'текст поста <=> имя' ->", (f_time - s_time)

Время сохранения новых отношений 'текст поста <=> имя' -> 20405


Когда все связи сохранены, мы можем осуществить поиск. В случае пополнения данных, необходимо находить все связи при добавлении их в базу данных "имя фамилия" и так же кешировать.

### Поиск

Ход действий таков:
1. Сделать запрос в базу данных, что бы достать все свзязи в которых имя совпадает с запрашиваемым
2. Получить id групп из результата (пункт 1)
3. По найденным id групп мы теперь можем получить посты, тексты которых содержат запрашиваемые имена.

Для демонстрации было взято предположительно самое популярное имя <i>"Donald Trump"</i> (имя, которое не сходит с уст СМИ), так как большинство сохраненных постов относятся к домену "Политика".

In [18]:
# Список имен для поиска
searched_names = [
    "John Kerry",
    "Paul Reichler",
    "Donald Trump"
]

def find_messages_by_fl_name(fl_name):
    finded_relations = fdb.get_name_relations_by_fl(fl_name=fl_name)
    messages = []
    finded_post_ids = set() # Необходимо, что бы учитывать ранее найденные посты

    for relation in finded_relations:
        post_id = relation['_source']['post_id']

        if not post_id in finded_post_ids:
            finded_post_ids.add(post_id)
            
            message = fdb.get_post_by_id(post_id)['_source']['message']
            messages.append(message)

    return messages


for name in searched_names:
    messages = find_messages_by_fl_name(name)
    
    print "Имя [", name, "] найдено соответстивий (сообщений):", len(messages)

Имя [ John Kerry ] найдено соответстивий (сообщений): 19
Имя [ Paul Reichler ] найдено соответстивий (сообщений): 10
Имя [ Donald Trump ] найдено соответстивий (сообщений): 227


## Удаление стоп-слов

Так же, частью подготовки документов (постов с facebook) является удаление стоп-слов (stop-words), это слова которые не несут никакой значимой информации и никак не отображают тему текста. В русском это были бы слова "а", "и", "или", "в" и тд. Эти слова встречаются в всюду и избавившись от них мы сделаем наши данные чище.

Список стоп-слов для английского языка лежат в отдельном файле с иминем "stop-words.txt". 

In [19]:
def load_stop_words(file_name):
    words = set()

    f = open(file_name, 'r')

    for line in f:
        words.add(line.strip())

    f.close()

    return words

stop_words = load_stop_words("stop_words.txt")
print "Размер словаря стоп-слов:", len(stop_words), "слова"

Размер словаря стоп-слов: 323 слова


Пакет <b>scikit-learn</b> уже содержит список стоп-слов для английского языка, и что бы не упустить ничего, объеденим их словарь с нашим (тот, который мы выгрузили с файла ранее)

In [20]:
from sklearn.feature_extraction import text 


stop_wrods = text.ENGLISH_STOP_WORDS.union(stop_words)
print "Размер объедененного словаря стоп-слов:", len(stop_words), "слова"

Размер объедененного словаря стоп-слов: 323 слова


## Токенизация данных

Добавим процедуру стемминга в <b>CountVectorizer</b> через наследование, создав новый класс <b>StemmedCountVectorizer</b>. Использован стиммер Портера из модуля <b>nltk</b>. Сам же стиммер будет использован после этапа токенизации и до формирования списка токенов по обработаному корпусу в вектооризаторе.

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from nltk.stem.porter import *


class StemmedCountVectorizer(CountVectorizer):
    def __init__(self):
        super(StemmedCountVectorizer, self).__init__()
        self.stemmer = PorterStemmer()  # проинициализируем стиммер Портера
        self.stop_words = stop_words    # проинициализируем словарь стоп-слов

    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc:(self.stemmer.stem(w) for w in analyzer(doc))


def create_vectorizer(): return StemmedCountVectorizer()

## Формирование списка топ 100 важных токенов по каждой категории

Выполним запрос к базе данных, что бы вытащить списко всех категорий (тех, что мы записывали в базу данных)

In [22]:
domains = fdb.get_all_domains()

Теперь создадим словарь, где ключем будет служить имя домена, а значением посты по этому домену.

In [23]:
s_time = time()
posts = dict([(domain, fdb.get_messages_by_domain(domain)) for domain in domains])
f_time = time()

print "Время получения постов с базы данных ->", (f_time - s_time)

Время получения постов с базы данных -> 774


In [24]:
for k, v in posts.items():
    print "Кол-во постов [", "{0: <10}".format(str(k)), "]:", len(v)

Кол-во постов [ pokemon_go ]: 349
Кол-во постов [ music      ]: 7986
Кол-во постов [ politics   ]: 7304
Кол-во постов [ tech       ]: 11986
Кол-во постов [ sport      ]: 1117
Кол-во постов [ finances   ]: 7592


Что бы работать с каждым классом независимо, создаем для каждого класса свой векторизатор, так как векторизатор кеширует список токенов трансформированного корпуса. Векторизаторы (в данном случае CountVectorizer) будут хранится в словаре, под ключем имени категории.

In [25]:
vs = dict([(domain, create_vectorizer()) for domain in domains])

Теперь векторизируем с помощью соответствующего векторизатора, соответствующий корпус.

In [26]:
# Словарь => key="имя_домена", value="матрица векторов соответствующего корпуса"
xs = dict([(domain, vs[domain].fit_transform(posts[domain])) for domain in domains])

In [27]:
for k, v in vs.items():
    print "Размеры словарей [", "{0: <10}".format(str(k)), "]:", len(v.vocabulary_)

Размеры словарей [ pokemon_go ]: 1085
Размеры словарей [ music      ]: 5997
Размеры словарей [ politics   ]: 38471
Размеры словарей [ tech       ]: 11110
Размеры словарей [ sport      ]: 3144
Размеры словарей [ finances   ]: 9372


Используя разряженную матрицу векторов, полученную от <b>CountVectorizer</b>, посчитаем кол-во вхождений каждого слова во всех документах (постах)

In [28]:
def find_top_tokens(matrix, vocabulary, amount, domain_name=None):
    s_time = time()
    
    counts = [(word, matrix.getcol(col_num).sum()) for word, col_num in vocabulary.items()]
    tokens = sorted(counts, key = lambda x: -x[1])[:min(amount, len(counts))]
    tokens_res = []
    
    for i in range(len(tokens)):
        token = tokens[i]
        tokens_res.append((token[0], i))
    
    f_time = time()
    
    if domain_name == None:
        print "Время поиска [", amount, "] популярных токенов ->", (f_time - s_time)
    else:
        print "Время поиска [", "{0: <10}".format(domain_name), "-", amount, "] популярных токенов ->", (f_time - s_time)
        
    return tokens_res

In [29]:
top_words_dict = dict([(d, find_top_tokens(xs[d], vs[d].vocabulary_, 100, d)) for d in domains])

Время поиска [ tech       - 100 ] популярных токенов -> 8104
Время поиска [ music      - 100 ] популярных токенов -> 2573
Время поиска [ finances   - 100 ] популярных токенов -> 4576
Время поиска [ politics   - 100 ] популярных токенов -> 71886
Время поиска [ sport      - 100 ] популярных токенов -> 410
Время поиска [ pokemon_go - 100 ] популярных токенов -> 74


Создадим из каждого списка сеты, что бы воспользоватся возможностями множеств для решения задачи классификации.

In [30]:
top_words_set = dict([(d, set(w[0] for w in top_words_dict[d])) for d in domains])
len(top_words_set)

6

In [35]:
for k, v in top_words_dict.items():
    print "Несколько популярных слов со списка [", "{0: <10}".format(str(k)), "]:"
    print v[:min(len(v), 5)], "...\n"

Несколько популярных слов со списка [ pokemon_go ]:
[(u'pokemon', 0), (u'app', 1), (u'gym', 2), (u'catch', 3), (u'just', 4)] ...

Несколько популярных слов со списка [ music      ]:
[(u'new', 0), (u'mtv', 1), (u'like', 2), (u'just', 3), (u'look', 4)] ...

Несколько популярных слов со списка [ politics   ]:
[(u'china', 0), (u'trump', 1), (u'news', 2), (u'say', 3), (u'donald', 4)] ...

Несколько популярных слов со списка [ tech       ]:
[(u'new', 0), (u'ti', 1), (u'make', 2), (u'just', 3), (u'like', 4)] ...

Несколько популярных слов со списка [ sport      ]:
[(u'team', 0), (u'sport', 1), (u'like', 2), (u'best', 3), (u'player', 4)] ...

Несколько популярных слов со списка [ finances   ]:
[(u'cnnmon', 0), (u'year', 1), (u'cnntech', 2), (u'money', 3), (u'make', 4)] ...



## Категоризация новых текстов (определение домена текста)

Алгоритм определения домена (класса) текста:
1. Найти пересечение множества слова текста с множествами топ-100 слов по каждой категории
2. Посмотреть колличественно на размер пересечений и склонится в сторону большего
3. Если категории равны по размеру пересечений, найти вес каждого пересечения отталкиваясь от позиции кажлого слова пересечения в списке топ-100 слов по каждой категории, т.е. чем выше в списке, тем вес слова больше.
4. В случае, если и тут несколько категорий равносильны, сделать выбор сравнив имена категорий

In [32]:
import operator

# Поиск элементов в итерируемых коллекциях
def find_elm(iterable, predicate_):
    for v in iterable:
        if predicate_(v):
            return v

    return None

# Для поиска категории к которой относится текст (функция find_domain(txt)), 
# нам понядобится один вектооризатор
fd_vec = create_vectorizer()

def find_domain(txt):
    # Если список категорий пуст, то нечего искать
    if len(domains) == 0:
        return None
    
    # Так векторизатор перезапишет свое состояние и можно будет получить список слов текста,
    # в которых удалены стоп слова и выполнен стемминг
    fd_vec.fit_transform([txt])
    
    # Теперь используя словарь данного текста - превратим его в сет,
    # что необходимо для последующих манипуляций
    words = [item[0] for item in fd_vec.vocabulary_.items()]
    
    # Сет слов текста
    words_set = set(words)
    
    # Тут мы будем хранить пересечения словаря этого текста со словарями 
    # по каждой категории
    res_buffer = []
    
    words_intersection_set = None
    
    # Сюда будем складывать результаты пересечений множеств слов текста и топ 100 по категориям
    isds = []
    
    # Для каждого домена со списка
    for domain in domains:
        # Достаем для данного домена сет топ 100-та слов
        domain_top_words_set = top_words_set[domain]
        
        # Находим пересечение словарей по категориям и словаря текста
        words_intersection_set = words_set.intersection(domain_top_words_set)    
        words_intersection_len = len(words_intersection_set)
        
        isds.append((domain, words_intersection_set))
        res_buffer.append((domain, words_intersection_len))
    
    if len(res_buffer) == 1:
        return res_buffer[0][0]

    # Находим категорию с наибольшим кол-во пересечений
    top_tuple = max(res_buffer, key=lambda t: t[1])
    # Максимальное кол-во пересечений
    max_count = (top_tuple)[1]
    
    # Формируем список из категорий с которыми одинаковое кол-во пересечений
    res = [name for name, count in res_buffer if count == max_count]
    
    if len(res) == 0:
        return None
    
    if len(res) == 1:
        return res[0]
    
    # Имя результирующей категории (домена)
    domain_res = None
    # Вес результирующей категории (домена)
    domain_vol = -1
    
    for domain in res:
        # Находим вес для каждого слова в пересечениях с разными категориями
        # чтобы по весу сказать какая категория преобладает
        top_words = top_words_dict[domain]
        
        # Вес
        vol = 0
        
        words_intersection_set = find_elm(isds, lambda e: e[0] == domain)[1]
        
        for w in words_intersection_set:
            vol_tmp = find_elm(top_words, lambda e: e[0] == w)[1]
            
            if vol_tmp != None:
                vol += vol_tmp
        
        # В случае, если и вес одинаков, сравниваем имена груп
        # и выбор делаем в сторону большей
        if vol > domain_vol:
            domain_res = domain
            domain_vol = vol
        elif vol == domain_vol:
            if domain > domain_res:
                domain_res = domain
                domain_vol = vol
    
    return domain_res

Теперь мы можем провести тестирование на заведомо известных документах, которые были классифицированы вручную.

In [33]:
docs_test = [
    ("politics","As islanders look forward to next year, many say they hope — and even expect — Hillary Clinton to spend at least part of her summer vacations on Martha's Vineyard if she becomes president."),
    ("politics","According to the Pentagon, Special Operation Forces targeted and killed ISIS leader Hafiz Sayed Khan in Afghanistan."),
    
    ("tech", "The Pentagon says staff can still play the game on their personal phones."),
    ("tech", "Intel's making drones now."),
    ("tech", "Samsung Mobile really pulled out all the stops on the new Galaxy Note 7"),
    ("tech", "VR anywhere? Nvidia's GTX 10-series GPUs in notebooks make that a reality."),
    
    ("music", "'The Second Amendment was put in there not just so we can go shoot skeet or go shoot trap. It was put in so we could defend our First Amendment, the freedom of speech, and also to defend ourselves against our own government,' says US Olympic skeet shooter Kim Rhode."),
    ("music", "Get ready... Britney Spears is performing at the 2016 Video Music Awards on August 28!"),
    
    ("pokemon_go", "Any idea why the 'ar' feature is always on when battling in gyms but not when catching Pokemon? (Ar option is off)"),
    ("pokemon_go", "The best thing just happened to me.One of my fav pokemon is arcanine. I have a 640 growlithe with 39 candy. I was calculating my pokemom ivs when a growlithe appeared in my nearby list....so i put on shoes and went to track him down....i found him and while i was catching him i was thinking to myself that ill have 43 candies(3 and transfer) and i was like imagine if i find 2 more and i get an arcanine....after i caught him my 5k egg hatched and i was like no way it will be a growlithe and it was!!!!! A 746 growlithe which boosted me up to 60 candies ready to evolve !!!!!!!!!!!"),
    ("pokemon_go", "So i have just checked a different iv calculator. I have been using silph road for some time and i just tried pokeassistant. They didnt match , i dont know what to think. There doesnt seem to be an accurate iv calculator"),
    
    ("finances", "Ronald Reagan's daughter Patti Davis is citing her father's shooting as evidence that comments like Donald J. Trump's recent blast against Hillary Clinton have real-world consequences."),
    ("finances", "If Hillary Clinton wins in November, she will be the first former secretary of state to take over the Oval Office since 1857."),
    ("finances", "Even if you're mentally prepared for a high priced ticket, some of the added travel costs associated with flying might throw you for a loop."),
    ("finances", "Under Armour founder and CEO Kevin Plank says if he's able to build his dream for the company in Baltimore it will create 'thousands to tens of thousands' of jobs, and says 'we see a vision here for us to make something greater'."), 
    ("finances", "You never have to pay for student loan help. If you're eligible, your loan servicer will offer you lower monthly payments or debt forgiveness of your federal student loans for free.")
]
    
res_domains_true = []
res_domains_pred = []

for domain_true, doc in docs_test:
    domain_pred = find_domain(doc)  
    
    res_domains_true.append(domains.index(domain_true))
    res_domains_pred.append(domains.index(domain_pred))
    
    print doc[0:45] + "...", "->", domain_pred
    
print
print "domains true:", res_domains_true
print "domains pred:", res_domains_pred

As islanders look forward to next year, many ... -> finances
According to the Pentagon, Special Operation ... -> politics
The Pentagon says staff can still play the ga... -> tech
Intel's making drones now.... -> politics
Samsung Mobile really pulled out all the stop... -> tech
VR anywhere? Nvidia's GTX 10-series GPUs in n... -> tech
'The Second Amendment was put in there not ju... -> politics
Get ready... Britney Spears is performing at ... -> music
Any idea why the 'ar' feature is always on wh... -> pokemon_go
The best thing just happened to me.One of my ... -> pokemon_go
So i have just checked a different iv calcula... -> pokemon_go
Ronald Reagan's daughter Patti Davis is citin... -> finances
If Hillary Clinton wins in November, she will... -> finances
Even if you're mentally prepared for a high p... -> finances
Under Armour founder and CEO Kevin Plank says... -> finances
You never have to pay for student loan help. ... -> finances

domains true: [3, 3, 0, 0, 0, 0, 1, 1, 5, 5, 5, 2, 

## Оценка качества классификации

Для оценки качества алгоритма классификации была выбрана <b>F1-score</b>, которая является относитеьно устойчивой под различные входные данные.

<p><b>F1-score</b> - это гармоническое среднее между точностью и полнотой. В данном случае точность и полнота равноценны.</p>

<p><b>Точность</b> <i>(precision)</i> – это доля документов действительно принадлежащих данному классу относительно всех документов которые система отнесла к этому классу.</p>
<p><b>Полнота</b> <i>(recall)</i> – это доля найденных классфикатором документов принадлежащих классу относительно всех документов этого класса в тестовой выборке.</p>

$$F1=2* \frac{Precision*Recall}{Precision+Recall}$$

В состав библиотеки <b>scikit-learn</b> входит реализация <b>F1-score</b> в виде функции <b>f1_score</b> из модуля <b>sklearn.metrics</b>

Значение параметра <b>average</b> дает возможность по разному оценивать данные, к примеру в ситуациях, когда данных одного конкретного класса больше нежели остальных. Определяет некую стратегию подсчета F1-score:
<p><b>weighted</b> - будет посчитано F1-score для каждого класса, а результатом будет их среднее значение, так же будет учтено кол-во эелементов по каждому классу, чем их больше тем важнее они воспринимаются</p>
<p><b>micro</b> - общая оценка где рассматривается глобальное показатели</p>
<p><b>macro</b> - будет посчитано F1-score для каждого класса, а результатом будет их среднее значение</p>

In [34]:
from sklearn.metrics import f1_score

print "F1-score (micro)   :", f1_score(res_domains_true, res_domains_pred, average="micro")
print "F1-score (macro)   :", f1_score(res_domains_true, res_domains_pred, average="macro")
print "F1-score (weighted):", f1_score(res_domains_true, res_domains_pred, average="weighted")
print "F1-score (None)    :", f1_score(res_domains_true, res_domains_pred, average=None)

F1-score (micro)   : 0.8125
F1-score (macro)   : 0.76658008658
F1-score (weighted): 0.81920995671
F1-score (None)    : [ 0.85714286  0.66666667  0.90909091  0.4         1.        ]
